In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from model import Net
from dataset import Cifar10

In [4]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

DATASET_PATH = "cifar-10-batches-py/data_batch_1"
traindata = Cifar10(DATASET_PATH, transform)
trainloader = DataLoader(traindata, batch_size=5, shuffle=True, num_workers=2)

DATASET_PATH = "cifar-10-batches-py/data_batch_1"
testdata = Cifar10(DATASET_PATH, transform)
testloader = DataLoader(testdata, batch_size=5, shuffle=True, num_workers=2)

In [5]:
class_count = {}
for sample in traindata:
    label = traindata.classes[sample[1]]
    if label not in class_count:
        class_count[label] = 0
    class_count[label] += 1
class_count

{'frog': 1030,
 'truck': 981,
 'deer': 999,
 'automobile': 974,
 'bird': 1032,
 'horse': 1001,
 'ship': 1025,
 'cat': 1016,
 'dog': 937,
 'airplane': 1005}

In [6]:
cuda = torch.cuda.is_available()
cuda

True

In [8]:
device = torch.device('cuda' if cuda else 'cpu')
device

device(type='cuda')

In [9]:
import torch.optim as optim


net = Net()

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [10]:
net.train(trainloader, optimizer, criterion, epochs=38)

[1,  2000] loss: 2.215
[2,  2000] loss: 1.884
[3,  2000] loss: 1.636
[4,  2000] loss: 1.500
[5,  2000] loss: 1.402
[6,  2000] loss: 1.327
[7,  2000] loss: 1.253
[8,  2000] loss: 1.187
[9,  2000] loss: 1.111
[10,  2000] loss: 1.046
[11,  2000] loss: 0.967
[12,  2000] loss: 0.905
[13,  2000] loss: 0.822
[14,  2000] loss: 0.756
[15,  2000] loss: 0.696
[16,  2000] loss: 0.642
[17,  2000] loss: 0.574
[18,  2000] loss: 0.531
[19,  2000] loss: 0.465
[20,  2000] loss: 0.422
[21,  2000] loss: 0.402
[22,  2000] loss: 0.372
[23,  2000] loss: 0.352
[24,  2000] loss: 0.335
[25,  2000] loss: 0.318
[26,  2000] loss: 0.296
[27,  2000] loss: 0.268
[28,  2000] loss: 0.233
[29,  2000] loss: 0.232
[30,  2000] loss: 0.232
[31,  2000] loss: 0.224
[32,  2000] loss: 0.230
[33,  2000] loss: 0.218
[34,  2000] loss: 0.199
[35,  2000] loss: 0.211
[36,  2000] loss: 0.191
[37,  2000] loss: 0.152
[38,  2000] loss: 0.165
Finished Training


In [11]:
net.score(testloader)

Accuracy of the network on the 10000 test images: 95 %
